# Notebook de comparación de posiciones y visualización de resultados
## Autor: José Miguel Ramírez Sanz

In [45]:
# imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import math
import os
from datetime import datetime as dt
import pandas as pd
import pickle as pk
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# import de utilities de detectron
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

### Carga del modelo

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.999  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

### Posicion

In [3]:
#Posiciones versión 3
class Posicion():
    
    def __init__(self,x,y):
        self.nariz = [x[0],y[0]]
        self.hombroI=[x[5],y[5]]
        self.hombroD=[x[6],y[6]]
        self.cuello = self.calcularPuntoMedio(self.hombroI,self.hombroD)
        self.angCuelloSupI = self.calcularAuxAngulo(self.hombroI,self.cuello,self.nariz)
        self.angCuelloSupD = self.calcularAuxAngulo(self.hombroD,self.cuello,self.nariz)
        self.codoI = [x[7],y[7]]
        self.codoD = [x[8],y[8]]
        self.manoI=[x[9],y[9]]
        self.manoD = [x[10],y[10]]
        self.angCodoI = self.calcularAngulo(self.hombroI,self.codoI,self.manoI,0)
        self.angCodoD = self.calcularAngulo(self.hombroD,self.codoD,self.manoD,1)
        self.angHombroI = self.calcularAngulo(self.cuello,self.hombroI,self.codoI,0)
        self.angHombroD = self.calcularAngulo(self.cuello,self.hombroD,self.codoD,1)
        self.caderaI = [x[11],y[11]]
        self.caderaD = [x[12],y[12]]
        self.cadera = self.calcularPuntoMedio(self.caderaI,self.caderaD)
        self.rodillaI = [x[13],y[13]]
        self.rodillaD = [x[14],y[14]]
        self.angCaderaI = self.calcularAngulo(self.cadera,self.caderaI,self.rodillaI,0)
        self.angCaderaD = self.calcularAngulo(self.cadera,self.caderaD,self.rodillaD,1)
        self.angCaderaTorsoI = self.calcularAuxAngulo(self.cuello,self.cadera,self.caderaI)
        self.angCaderaTorsoD = self.calcularAuxAngulo(self.cuello,self.cadera,self.caderaD)
        self.tobilloI = [x[15],y[15]]
        self.tobilloD = [x[16],y[16]]
        self.angRodillaI = self.calcularAngulo(self.caderaI,self.rodillaI,self.tobilloI,0)
        self.angRodillaD = self.calcularAngulo(self.caderaD,self.rodillaD,self.tobilloD,1)

        
    def calcularPuntoMedio(self,p1,p2):
        return [(p1[0]+p2[0])/2,(p1[1]+p2[1])/2]
    
    def calcularAuxAngulo(self,p1,p2,p3):
        v1 = self.calcularVector(p1,p2)
        v2 = self.calcularVector(p3,p2)
        uv1 = v1 / np.linalg.norm(v1)
        uv2 = v2 / np.linalg.norm(v2)
        dp = np.dot(uv1, uv2)
        if dp > 1:
            dp=1
        elif dp < -1:
            dp=-1
        return math.degrees(np.arccos(dp))
    
    def calcularVector(self,p1,p2):
        return [p2[0]-p1[0],p2[1]-p1[1]]
    
    
    def calcularAngulo(self,p1,p2,p3,lado):
        flag=False
        ang = self.calcularAuxAngulo(p1,p2,p3)
        
        #Cuando la recta que une los dos primeros puntos es una línea vertical
        if (p2[0]-p1[0]) ==0:
            #lado derecho
            if lado:
                #Si la posicion está menos a la derecha se ha de cambiar el ángulo
                if p3[0]<p2[0]:
                    flag=True
            #lado izquierdo
            else:
                #Si la posicion está menos a la izquierda se ha de cambiar el ángulo
                if p3[0]>p2[0]:
                    flag=True
            #Cambio del ángulo
            if flag:
                ang = 360 - ang
        else:
            y = ((p2[1]-p1[1])*(p3[0]-p1[0])/(p2[0]-p1[0]))+p1[1]
            #Si está la tercera parte por encima de la recta que hacen las dos primera entonces se cambia el angulo
            if p3[1] > y:
                ang = 360 - ang
        
        return ang

### Comparación

In [ ]:
def compararPosiciones(pos1,pos2,pesos={"brazos":1,"piernas":1,"torso":1}):
    zonas={"brazos":["angCodo","angHombro"],"piernas":["angRodilla","angCadera"],"torso":["angCaderaTorso","angCuelloSup"]}
    res=0
    total=0
    result = {}
    
    #Se recoge el peso total
    for i in pesos:
        if i not in zonas:
            raise Exception("No se puede dar peso a una zona que no esté definida")
        total+=pesos[i]
        
    #Se recorren los distintos tipos de zonas y se les aplica el peso a la comparación
    for i in zonas:
        result[i]=comparacionZona(pos1,pos2,zonas[i])
        res+=(pesos[i]/total)*result[i]
    
    porcentaje = res*100/180
    
    for i in result:
        result[i]=result[i]*100/180
    
    return res,100-porcentaje,result["brazos"],result["piernas"],result["torso"]

In [4]:
def comparacionZona(pos1,pos2,zonas):
    partes=["D","I"]
    res = 0.0
    for i in partes:
        for j in zonas:
            #aux es la diferencia entre los ángulos
            aux = abs(eval("pos1."+j+i)-eval("pos2."+j+i))
            #si la diferencia es mayor de 180 grados se coge el otro lado
            if aux > 180:
                res += (360-aux)
            else:
                res+=aux
    return res/(len(partes)*len(zonas))

### Funciones

In [5]:
def obtenerFrame(vid,nframe,path='../pruebas/videos-prueba/Videos/Josemi'):
    vc = cv2.VideoCapture(path+vid)

    if (vc.isOpened()==False):
        print("Error")
    else:
        for i in range(nframe):
            ret,frame = vc.read()

        o = predictor(frame)
        v = Visualizer(frame[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        v2 = v.draw_instance_predictions(o["instances"].to("cpu"))
        imVisualizer = cv2.cvtColor(v2.get_image()[:, :, ::-1],cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(18, 16))
        plt.imshow(imVisualizer)
        plt.show()

        pkP = o.get("instances").pred_keypoints
        if len(pkP)>0:
            x = pkP[0][:,0].cpu().numpy()
            y = pkP[0][:,1].cpu().numpy()
    return Posicion(x,y)

In [6]:
def obtenerPosiciones(vid):
    pos=[]
    vc = cv2.VideoCapture(vid)

    if (vc.isOpened()==False):
        print("Error")

    while(vc.isOpened()):
        ret,frame = vc.read()
        if ret == True:
            o = predictor(frame)
            pkP = o.get("instances").pred_keypoints
            if len(pkP)>0 and len(pkP[0])==17:
                x = pkP[0][:,0].cpu().numpy()
                y = pkP[0][:,1].cpu().numpy()
                pos.append(Posicion(x,y))
            else:
                pos.append(None)
        else:
            break
        print("Frame " + str(i),end="\r")
        i+=1

    vc.release()
    cv2.destroyAllWindows()
    
    return pos

In [131]:
def imprimeVelocimetro(valor,titulo, colorTitulo="darkblue", colorBarra="darkblue"):
    fig = go.Indicator(
        domain = {'x': [0, 1], 'y': [0, 1]},
        value = valor,
        mode = "gauge+number",
        title = {'text': titulo,'font':{'color': colorTitulo, 'family': "Arial","size":20}},
        delta = {'reference': None},
        gauge = {'axis': {'range': [None, 100]},
                 'bar': {'color': colorBarra},
                 'steps' : [
                     {'range': [0, 33], 'color': 'lightcoral'},
                    {'range': [33, 66], 'color': 'khaki'},
                    {'range': [66, 100], 'color': 'palegreen'}],
                 'threshold' : {'line': {'color': "red", 'width': 4}}})

    return fig

In [136]:
def imprimeVelocimetros(media,maximo,minimo,colorTitulo="darkblue", colorBarra="darkblue"):

    fig = make_subplots(
            rows=1, cols=3,
            specs=[[{"type": "domain"}, {"type": "domain"},{"type": "domain"}]],
        )

    fig.add_trace(imprimeVelocimetro(media,"Porcentaje exactitud medio",colorTitulo, colorBarra),
                  row=1, col=1)

    fig.add_trace(imprimeVelocimetro(maximo,"Porcentaje exactitud máximo",colorTitulo, colorBarra),
                  row=1, col=2)

    fig.add_trace(imprimeVelocimetro(minimo,"Porcentaje exactitud mínimo", colorTitulo, colorBarra),
                  row=1, col=3)

    fig.update_layout(paper_bgcolor = "white", font = {'color': colorTitulo, 'family': "Arial","size":15})
    
    fig.show()

In [138]:
imprimeVelocimetros(80,60,100)
imprimeVelocimetros(80,60,90)